In [ ]:
import os
from datetime import datetime


import numpy as np
import pandas as pd

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())


In [ ]:
df = pd.read_excel(os.environ.get("ODSPEP_FILE_URL"), dtype=str).replace(
    [np.nan, ""], None
)

In [ ]:
df.sample(5)


### Description générale

In [ ]:
df.shape[0]

### analyse

* orienté services
* 696 services dans le fichier en théorie
* dénormalisation des autres dimensions (mise à plat) -> `ID_RES` dupliqués
* jointure pour obtenir les "libellés" à partir des codes
* ok : lat/lon, adresse sous norme AFNOR, code insee
* ok : horaires d'ouverture au format maison
* ok : contacts de la structure (champs *_CTC), tel très remplis
* ko : pas de siret
* ko : pas de date de maj
* ko : pas de champs pour mapper la typologie de structure

### documentation

* les champs `*_ADR` correspondent à l'adresse de la structure (et non du service)
* pour obtenir la zone de diffusion du service, regrouper le champs `COM_SERVICE` pour un même `ID_RES`


### Répartition géographique ?

* à prendre avec des pincettes car le fichier est dénormalisé
* d'autres départements que ceux initialement demandés sont présent car un service dans le cd35 par exemple peut avoir une zone de diffusion plus grande que le département.

In [ ]:
df[["DEPT_SERVICE", "Departement Service"]].value_counts().to_frame()

## Structures

### id

In [ ]:
df.shape[0]


In [ ]:
ressources_df = df.iloc[
    :,
    (df.columns == "ID_RES")
    | (df.columns == "STRUCTURE")
    | (df.columns == "LIBELLE_SERVICE")
    | (df.columns == "DESCRIPTION_SERVICE")
    | (df.columns == "DATE DERNIERE MAJ")
    | df.columns.str.endswith("_ADR")
    | df.columns.str.endswith("_PHY")
    | df.columns.str.endswith("_CTC"),
]
ressources_df = ressources_df.drop_duplicates(subset="ID_RES")

In [ ]:
ressources_df.info()


In [ ]:
ressources_df.shape[0]


In [ ]:
ressources_df.STRUCTURE.duplicated(keep=False).sum()


### siret

### rna

### nom

In [ ]:
ressources_df[["STRUCTURE", "LIBELLE_SERVICE"]].sample(20)


In [ ]:
ressources_df.STRUCTURE.isna().sum()


In [ ]:
ressources_df.LIBELLE_SERVICE.isna().sum()


### commune

In [ ]:
ressources_df.LIBELLE_COMMUNE_ADR.isna().sum()


In [ ]:
ressources_df.LIBELLE_COMMUNE_ADR.sample(20).to_frame()


### code_postal

In [ ]:
ressources_df.CODE_POSTAL_ADR.isna().sum()


In [ ]:
ressources_df.CODE_POSTAL_ADR.sample(20).to_frame()


### code_insee

In [ ]:
ressources_df.CODE_COMMUNE_ADR.sample(20).to_frame()


### adresse

In [ ]:
ressources_df.L4_NUMERO_LIB_VOIE_ADR.isna().sum()


### complement_adresse

### longitude

### latitude

### typologie

### telephone

In [ ]:
contacts_df = df[
    [
        "ID_RES",
        "ID_CTC",
        "TEL_1_CTC",
        "TEL_2_CTC",
        "FAX_CTC",
        "SITE_INTERNET_CTC",
        "MAIL_CTC",
    ]
]
contacts_df = contacts_df.drop_duplicates()

### courriel

### site_web

### presentation_resume

### presentation_detail

### date_maj

In [ ]:
df["DATE DERNIERE MAJ"].value_counts()


### structure_parente

In [ ]:
ressources_df.STRUCTURE.duplicated(keep=False).sum()


### lien_source

### horaires_ouverture

In [ ]:
horaires_df = df.iloc[
    :,
    df.columns.str.contains("ID_RES|COMMENTAIRES_HORAIRE_RSP")
    | df.columns.str.endswith("_HOR"),
]
horaires_df = horaires_df.drop_duplicates()
horaires_df = horaires_df.dropna(subset=["JOUR_HOR"])
horaires_df.shape[0]

In [ ]:
horaires_df.sample(10)


### accessibilite

### labels_nationaux

### labels_autres

### thematiques

In [ ]:
familles_df = df[["ID_RES", "CODE_FAM", "FamilleBesoin"]]
familles_df = familles_df.drop_duplicates()


In [ ]:
familles_df.ID_RES.value_counts().to_frame().head(20)


In [ ]:
familles_df[["CODE_FAM", "FamilleBesoin"]].value_counts().to_frame()


In [ ]:
categories_df = df[["ID_RES", "CODE_CAT", "Besoin"]]
categories_df = categories_df.drop_duplicates()


In [ ]:
categories_df.ID_RES.value_counts().to_frame().head(20)


In [ ]:
categories_df[["CODE_CAT", "Besoin"]].value_counts().to_frame()


In [ ]:
sous_categories_df = df[["ID_RES", "CODE_SSC", "Sous besoin"]]
sous_categories_df = sous_categories_df.drop_duplicates()


In [ ]:
sous_categories_df.ID_RES.value_counts().to_frame().head(20)


In [ ]:
sous_categories_df[["CODE_SSC", "Sous besoin"]].value_counts().to_frame()


In [ ]:
all_cat_df = df[
    [
        "ID_RES",
        "CODE_FAM",
        "CODE_CAT",
        "CODE_SSC",
        "FamilleBesoin",
        "Besoin",
        "Sous besoin",
    ]
]
all_cat_df.drop_duplicates()
all_cat_df.shape[0]

In [ ]:
all_cat_df["Besoin"].drop_duplicates().to_list()


# Services

### id

### structure_id

### nom

In [ ]:
ressources_df.DESCRIPTION_SERVICE.map(len)


### Taux de remplissage des champs

In [ ]:
def compute_field_occupancy_rates(df):
    return ((1 - df.isnull().sum() / df.shape[0]) * 100).sort_values(ascending=False)

In [ ]:
compute_field_occupancy_rates(df).to_frame()

Typologie de structure

In [ ]:
from data_inclusion.schema import models

categories_flags_services_df = (
    services_df.STRUCTURE.str.lower()
    .apply(
        lambda s: {
            models.Typologie.CAF.value: "caf" in s.split()
            or ("caisse" in s and "allocation" in s and "fami" in s),
            models.Typologie.CC.value: "communaut" in s
            and "commune" in s
            and "maternelle" not in s,
            models.Typologie.ASSO.value: "association" in s.split(),
            models.Typologie.CCAS.value: "ccas" in s.split()
            or "social" in s
            and "action" in s,
            models.Typologie.CHRS.value: "chrs" in s.split()
            or ("bergement" in s and "insertion" in s),
            models.Typologie.RS_FJT.value: ("sidence" in s and "social" in s)
            or "fjt" in s
            or ("foyer" in s and "jeune" in s and "travail" in s),
            models.Typologie.CS.value: "centre social" in s,
            models.Typologie.MDS.value: "maison" in s and "solidarit" in s,
            models.Typologie.ML.value: "mission" in s and "local" in s,
            models.Typologie.MDPH.value: "maison" in s and "handic" in s,
        }
    )
    .apply(pd.Series)
    .assign(
        na=lambda df: df.apply(
            lambda row: ~row.any(), axis="columns", result_type="expand"
        )
    )
)

In [ ]:
categories_flags_services_df.iloc[:, categories_flags_services_df.columns != "na"].any(
    axis="columns"
).sum()

In [ ]:
categories_flags_services_df.sum().sort_values(ascending=False).plot(
    kind="bar", grid=True, rot=35, figsize=(20, 8)
)


In [ ]:
df.iloc[:, df.columns.str.endswith("_CTC")].sample(10)

Champ `type service partenaire` pour extraire un type de structure ?

In [ ]:
df.drop_duplicates(subset="ID_RES")[
    "type service partenaire "
].value_counts().to_frame()

### Nombre de structures sous-jacentes

In [ ]:
df.STRUCTURE.nunique()

Aperçu de la distribution du nombres de services par structure

Pour la majorité : un service par structure

In [ ]:
df.drop_duplicates("ID_RES").STRUCTURE.value_counts().head(10).to_frame()

### Siretisation automatique

In [ ]:
import os
import importlib

import dotenv
from tqdm.auto import tqdm
import sqlalchemy as sqla

from data_inclusion.tasks import siretisation

tqdm.pandas()

# reload siretisation without restarting the entire kernel
importlib.reload(siretisation)

dotenv.load_dotenv(dotenv.find_dotenv())

engine = sqla.create_engine(os.environ["SIRENE_DATABASE_URL"])

structures_df = df.drop_duplicates("ID_RES")

establishments_df = structures_df.progress_apply(
    lambda row: siretisation.search_establishment(
        nom=row.STRUCTURE,
        adresse=row.L4_NUMERO_LIB_VOIE_ADR,
        code_insee=row.CODE_COMMUNE_ADR,
        latitude=row.LATITUDE_ADR,
        longitude=row.LONGITUDE_ADR,
        engine=engine,
    )
    or {},
    axis="columns",
    result_type="expand",
)

structures_siretisees_df = pd.merge(
    df,
    establishments_df,
    how="left",
    right_index=True,
    left_index=True,
)

#### Résultats de la siretisation automatique

% de structures siretisées

In [ ]:
structures_siretisees_df.drop_duplicates(
    "STRUCTURE"
).siret.notna().sum() * 100 / structures_siretisees_df.drop_duplicates(
    "STRUCTURE"
).shape[
    0
]

% de services avec structures siretisées

Certaines structures ont un nombre important de services.

In [ ]:
structures_siretisees_df.siret.notna().sum() * 100 / structures_siretisees_df.shape[0]

établissement vs structures

In [ ]:
structures_siretisees_df[structures_siretisees_df.siret.notna()][
    ["ID_RES", "STRUCTURE", "siret", "name"]
]